In [1]:
# set module
import pandas as pd


from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
## hugging face의 transformer 다운
# !pip install transformers==4.28.0

In [ ]:
"""
주어진 오류는 Trainer와 PyTorch를 함께 사용할 때 accelerate 패키지가 필요하다는 내용입니다. accelerate는 Hugging Face의 Trainer와 PyTorch를 함께 사용할 때 모델 학습을 가속화하기 위해 설계된 패키지입니다.
"""
# !pip install git+https://github.com/huggingface/accelerate

In [3]:
import os

new_dir = "/content/drive/MyDrive"
os.chdir(new_dir)  # 현재 작업 디렉토리 변경
current_dir = os.getcwd()
print("현재 작업 디렉토리:", current_dir)



현재 작업 디렉토리: /content/drive/MyDrive


In [5]:
from transformers import RobertaTokenizer, RobertaForSequenceClassification
from transformers import Trainer
from transformers import TrainingArguments
from sklearn.metrics import accuracy_score
import torch

# GPU를 사용하려면 다음 코드를 실행하세요.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# RoBERTa 토크나이저와 모델을 불러옵니다.
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaForSequenceClassification.from_pretrained('roberta-base')

class roberta_train():
  def __init__(self, path, filename):
    self.dataset = None
    self.path = path
    self.filename = filename
    self.train_dataset = None

  # 데이터셋을 로드하고 토큰화합니다.
  def load_dataset(self):
      # TODO: 데이터셋을 로드하는 코드를 작성하세요.
      self.dataset = pd.read_csv(self.path + self.filename)
      return self.dataset
 

  def tokenize_dataset(self):
      tokenized = tokenizer(self.dataset['script'].tolist(), padding='max_length', truncation=True, max_length=128)
      tokenized['label'] = self.dataset['label']
      self.train_dataset = tokenized
      return self.train_dataset



# 데이터셋을 PyTorch의 Dataset 형식으로 변환합니다.
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)




Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'roberta.pooler.dense.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'roberta.pooler.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.

In [ ]:
# train_dataset = Dataset(train_dataset, train_dataset.label)

# # Trainer 및 TrainingArguments를 설정합니다.
# training_args = TrainingArguments(
#     output_dir='./results',
#     num_train_epochs=3,
#     per_device_train_batch_size=16,
#     per_device_eval_batch_size=16,
#     warmup_steps=500,
#     weight_decay=0.01,
#     logging_dir='./logs',
#     logging_steps=10
# )

# # Trainer 객체를 생성하고 fine-tuning을 실행합니다.
# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=train_dataset,
#     compute_metrics=lambda pred: {'accuracy': accuracy_score(pred.label_ids, pred.predictions.argmax(-1))}
# )

# trainer.train()
# from sklearn.model_selection import KFold
# train_dataset = Dataset(train_dataset, train_dataset.label)


# # 데이터셋을 K-fold로 나누기 위해 KFold 객체를 생성합니다.
# kfold = KFold(n_splits=5, shuffle=True)

# # K-fold를 반복하여 학습을 수행합니다.
# for fold, (train_indices, val_indices) in enumerate(kfold.split(train_dataset)):
#     # 현재 fold에 해당하는 데이터를 추출합니다.
#     train_data_fold = [train_dataset[i] for i in train_indices]
#     val_data_fold = [train_dataset[i] for i in val_indices]

#     # Trainer 및 TrainingArguments를 설정합니다.
#     training_args = TrainingArguments(
#         output_dir=f'./results_fold_{fold}',
#         num_train_epochs=3,
#         per_device_train_batch_size=16,
#         per_device_eval_batch_size=16,
#         warmup_steps=500,
#         weight_decay=0.01,
#         logging_dir=f'./logs_fold_{fold}',
#         logging_steps=10
#     )

#     # Trainer 객체를 생성하고 fine-tuning을 실행합니다.
#     trainer = Trainer(
#         model=model,
#         args=training_args,
#         train_dataset=train_data_fold,
#         eval_dataset=val_data_fold,
#         compute_metrics=lambda pred: {'accuracy': accuracy_score(pred.label_ids, pred.predictions.argmax(-1))}
#     )

#     trainer.train()
#     trainer.save_model("./saved_model")  # 모델을 지정한 경로에 저장합니다.

## cv 사용해서 이전보다 accuracy 높을 경우에 저장

In [ ]:
# from sklearn.model_selection import KFold
# from sklearn.metrics import accuracy_score
# import os

# train_dataset = Dataset(train_dataset, train_dataset.label)

# # 데이터셋을 K-fold로 나누기 위해 KFold 객체를 생성합니다.
# kfold = KFold(n_splits=5, shuffle=True)

# # K-fold를 반복하여 학습을 수행합니다.
# for fold, (train_indices, val_indices) in enumerate(kfold.split(train_dataset)):
#     # 현재 fold에 해당하는 데이터를 추출합니다.
#     train_data_fold = [train_dataset[i] for i in train_indices]
#     val_data_fold = [train_dataset[i] for i in val_indices]

#     # Trainer 및 TrainingArguments를 설정합니다.
#     output_dir = f'./content/drive/MyDrive/bert_train_project/results_fold_{fold}'
#     logging_dir = f'./content/drive/MyDrive/bert_train_project/logs_fold_{fold}'

#     training_args = TrainingArguments(
#         output_dir=output_dir,
#         num_train_epochs=30,
#         per_device_train_batch_size=16,
#         per_device_eval_batch_size=16,
#         warmup_steps=500,
#         weight_decay=0.01,
#         logging_dir=logging_dir,
#         logging_steps=10,
#         save_strategy='steps',  # 매 50개의 steps마다 모델을 저장하도록 설정합니다.
#         save_steps=50,
#         save_total_limit=10,  # 최근 10개의 모델만 유지하고 나머지는 자동으로 삭제합니다.
#         evaluation_strategy='steps',  # 매 50개의 steps마다 검증을 수행합니다.
#         eval_steps=50,
#         logging_first_step=True,  # 첫 번째 step에서도 로깅을 수행합니다.
#         disable_tqdm=True  # tqdm progress bar를 사용하지 않습니다.
#     )

#     # EarlyStopping 콜백을 정의합니다.
#     early_stopping_callback = EarlyStoppingCallback(early_stopping_patience=5)

#     # Trainer 객체를 생성하고 fine-tuning을 실행합니다.
#     trainer = Trainer(
#         model=model,
#         args=training_args,
#         train_dataset=train_data_fold,
#         eval_dataset=val_data_fold,
#         compute_metrics=lambda pred: {'accuracy': accuracy_score(pred.label_ids, pred.predictions.argmax(-1))},
#         callbacks=[early_stopping_callback]  # EarlyStopping 콜백을 추가합니다.
#     )

#     # fine-tuning을 실행하면서 모델을 저장
#     trainer.train()

#     # 모델 저장을 위해 저장된 로그 파일 중에서 최근 50개만 선택합니다.
#     log_files = sorted(os.listdir(logging_dir))
#     selected_logs = log_files[-50:]

#     # 최종 모델만 저장하고 이전 모델 및 로그는 삭제합니다.
#     os.makedirs(f"./content/drive/MyDrive/bert_train_project/saved_model_fold_{fold}", exist_ok=True)
#     for log in selected_logs:
#         shutil.move(os.path.join(logging_dir, log), os.path.join(output_dir, log))
#     trainer.save_model(f"./content/drive/MyDrive/bert_train_project/saved_model_fold_{fold}/final_model")


In [6]:
from transformers import TrainerCallback

## train_dataset 제작,
path = "bert_train_project/"
filename = "train.csv"
train_dat = roberta_train(path, filename)
show_data = train_dat.load_dataset()
train_dataset = train_dat.tokenize_dataset()


class EarlyStoppingCallback(TrainerCallback):
    def __init__(self, early_stopping_patience=5, metric_name="eval_loss"):
        self.early_stopping_patience = early_stopping_patience
        self.metric_name = metric_name
        self.best_metric = float("inf")
        self.counter = 0

    def on_evaluate(self, args, state, control, **kwargs):
        current_metric = state.log_history[-1].get(self.metric_name)
        if current_metric is not None and current_metric < self.best_metric:
            self.best_metric = current_metric
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.early_stopping_patience:
                control.should_training_stop = True


from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
import os
import shutil

train_dataset = Dataset(train_dataset, train_dataset.label)

# 데이터셋을 K-fold로 나누기 위해 KFold 객체를 생성합니다.
kfold = KFold(n_splits=5, shuffle=True)

# K-fold를 반복하여 학습을 수행합니다.
for fold, (train_indices, val_indices) in enumerate(kfold.split(train_dataset)):
    # 현재 fold에 해당하는 데이터를 추출합니다.
    train_data_fold = [train_dataset[i] for i in train_indices]
    val_data_fold = [train_dataset[i] for i in val_indices]

    # Trainer 및 TrainingArguments를 설정합니다.
    output_dir = f'bert_train_project/results_fold_{fold}'
    logging_dir = f'bert_train_project/logs_fold_{fold}'

    training_args = TrainingArguments(
        output_dir=output_dir,
        num_train_epochs=30,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        warmup_steps=500,
        weight_decay=0.01,
        logging_dir=logging_dir,
        logging_steps=100,
        save_strategy='steps',  # 매 100개의 steps마다 모델을 저장하도록 설정합니다.
        save_steps=100,
        save_total_limit=10,  # 최근 10개의 모델만 유지하고 나머지는 자동으로 삭제합니다.
        evaluation_strategy='steps',  # 매 50개의 steps마다 검증을 수행합니다.
        eval_steps=100,
        logging_first_step=True,  # 첫 번째 step에서도 로깅을 수행합니다.
        disable_tqdm=True  # tqdm progress bar를 사용하지 않습니다.
    )

    # EarlyStopping 콜백을 정의합니다.
    early_stopping_callback = EarlyStoppingCallback(early_stopping_patience=5)

    # Trainer 객체를 생성하고 fine-tuning을 실행합니다.
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_data_fold,
        eval_dataset=val_data_fold,
        compute_metrics=lambda pred: {'accuracy': accuracy_score(pred.label_ids, pred.predictions.argmax(-1))},
        callbacks=[early_stopping_callback]  # EarlyStopping 콜백을 추가합니다.
    )

    # fine-tuning을 실행하면서 모델을 저장
    trainer.train()

    # 모델 저장을 위해 저장된 로그 파일 중에서 최근 50개만 선택합니다.
    log_files = sorted(os.listdir(logging_dir))
    selected_logs = log_files[-50:]

    # 최종 모델만 저장하고 이전 모델 및 로그는 삭제합니다.
    os.makedirs(f"saved_model_fold_{fold}", exist_ok=True)
    for log in selected_logs:
        shutil.move(os.path.join(logging_dir, log), os.path.join(output_dir, log))
    trainer.save_model(f"saved_model_fold_{fold}/final_model")


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


{'loss': 0.6803, 'learning_rate': 1.0000000000000001e-07, 'epoch': 0.09}
{'loss': 0.4341, 'learning_rate': 1e-05, 'epoch': 9.09}
{'eval_loss': 0.0019315374083817005, 'eval_accuracy': 1.0, 'eval_runtime': 0.2769, 'eval_samples_per_second': 155.3, 'eval_steps_per_second': 10.835, 'epoch': 9.09}
{'loss': 0.001, 'learning_rate': 2e-05, 'epoch': 18.18}
{'eval_loss': 0.00028080641641281545, 'eval_accuracy': 1.0, 'eval_runtime': 0.2896, 'eval_samples_per_second': 148.5, 'eval_steps_per_second': 10.36, 'epoch': 18.18}
{'loss': 0.0003, 'learning_rate': 3e-05, 'epoch': 27.27}
{'eval_loss': 0.00011179783177794889, 'eval_accuracy': 1.0, 'eval_runtime': 0.298, 'eval_samples_per_second': 144.31, 'eval_steps_per_second': 10.068, 'epoch': 27.27}
{'train_runtime': 139.9265, 'train_samples_per_second': 36.233, 'train_steps_per_second': 2.358, 'train_loss': 0.13271587479746702, 'epoch': 30.0}


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


{'loss': 0.0002, 'learning_rate': 1.0000000000000001e-07, 'epoch': 0.09}
{'loss': 0.0111, 'learning_rate': 1e-05, 'epoch': 9.09}
{'eval_loss': 1.7296286387136206e-05, 'eval_accuracy': 1.0, 'eval_runtime': 0.2979, 'eval_samples_per_second': 144.364, 'eval_steps_per_second': 10.072, 'epoch': 9.09}
{'loss': 0.0009, 'learning_rate': 2e-05, 'epoch': 18.18}
{'eval_loss': 3.6649855701398337e-06, 'eval_accuracy': 1.0, 'eval_runtime': 0.2934, 'eval_samples_per_second': 146.533, 'eval_steps_per_second': 10.223, 'epoch': 18.18}
{'loss': 0.0, 'learning_rate': 3e-05, 'epoch': 27.27}
{'eval_loss': 1.5746700228191912e-06, 'eval_accuracy': 1.0, 'eval_runtime': 0.3005, 'eval_samples_per_second': 143.083, 'eval_steps_per_second': 9.983, 'epoch': 27.27}
{'train_runtime': 136.0331, 'train_samples_per_second': 37.27, 'train_steps_per_second': 2.426, 'train_loss': 0.0035838918283512943, 'epoch': 30.0}


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


{'loss': 0.0, 'learning_rate': 1.0000000000000001e-07, 'epoch': 0.09}
{'loss': 0.0, 'learning_rate': 1e-05, 'epoch': 9.09}
{'eval_loss': 4.825137693842407e-07, 'eval_accuracy': 1.0, 'eval_runtime': 0.2838, 'eval_samples_per_second': 147.993, 'eval_steps_per_second': 10.571, 'epoch': 9.09}
{'loss': 0.0, 'learning_rate': 2e-05, 'epoch': 18.18}
{'eval_loss': 1.2204759514133912e-07, 'eval_accuracy': 1.0, 'eval_runtime': 0.2921, 'eval_samples_per_second': 143.785, 'eval_steps_per_second': 10.27, 'epoch': 18.18}
{'loss': 0.0, 'learning_rate': 3e-05, 'epoch': 27.27}
{'eval_loss': 2.838316248698902e-09, 'eval_accuracy': 1.0, 'eval_runtime': 0.284, 'eval_samples_per_second': 147.884, 'eval_steps_per_second': 10.563, 'epoch': 27.27}
{'train_runtime': 138.2078, 'train_samples_per_second': 36.901, 'train_steps_per_second': 2.388, 'train_loss': 7.385011926861338e-07, 'epoch': 30.0}


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


{'loss': 0.0, 'learning_rate': 1.0000000000000001e-07, 'epoch': 0.09}
{'loss': 0.0, 'learning_rate': 1e-05, 'epoch': 9.09}
{'eval_loss': 0.0, 'eval_accuracy': 1.0, 'eval_runtime': 0.2849, 'eval_samples_per_second': 147.426, 'eval_steps_per_second': 10.53, 'epoch': 9.09}
{'loss': 0.0, 'learning_rate': 2e-05, 'epoch': 18.18}
{'eval_loss': 0.0, 'eval_accuracy': 1.0, 'eval_runtime': 0.2846, 'eval_samples_per_second': 147.595, 'eval_steps_per_second': 10.543, 'epoch': 18.18}
{'loss': 0.0, 'learning_rate': 3e-05, 'epoch': 27.27}
{'eval_loss': 0.0, 'eval_accuracy': 1.0, 'eval_runtime': 0.2946, 'eval_samples_per_second': 142.55, 'eval_steps_per_second': 10.182, 'epoch': 27.27}
{'train_runtime': 133.2894, 'train_samples_per_second': 38.263, 'train_steps_per_second': 2.476, 'train_loss': 2.967588743360645e-08, 'epoch': 30.0}


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


{'loss': 0.0, 'learning_rate': 1.0000000000000001e-07, 'epoch': 0.09}
{'loss': 0.0, 'learning_rate': 1e-05, 'epoch': 9.09}
{'eval_loss': 0.0, 'eval_accuracy': 1.0, 'eval_runtime': 0.2813, 'eval_samples_per_second': 149.305, 'eval_steps_per_second': 10.665, 'epoch': 9.09}
{'loss': 0.0, 'learning_rate': 2e-05, 'epoch': 18.18}
{'eval_loss': 0.0, 'eval_accuracy': 1.0, 'eval_runtime': 0.2916, 'eval_samples_per_second': 144.042, 'eval_steps_per_second': 10.289, 'epoch': 18.18}
{'loss': 0.0, 'learning_rate': 3e-05, 'epoch': 27.27}
{'eval_loss': 0.0, 'eval_accuracy': 1.0, 'eval_runtime': 0.2815, 'eval_samples_per_second': 149.175, 'eval_steps_per_second': 10.655, 'epoch': 27.27}
{'train_runtime': 145.3583, 'train_samples_per_second': 35.086, 'train_steps_per_second': 2.27, 'train_loss': 0.0, 'epoch': 30.0}


In [10]:
saved_model_path = "saved_model_fold_1/final_model"

tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
my_model = RobertaForSequenceClassification.from_pretrained(saved_model_path)

In [13]:
# from transformers import RobertaTokenizer, RobertaForSequenceClassification

# # Load the tokenizer
# tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

# # Load your fine-tuned model
# saved_model_path = "saved_model_fold_1/final_model"
# my_model = RobertaForSequenceClassification.from_pretrained(saved_model_path)
# # Example text
# text = """Good evening, ladies and gentlemen! Thank you for being here tonight. I want to talk to you about something that is at the core of our existence: human connection. It's a topic that has fascinated me for years, and I believe it holds the key to our personal growth, happiness, and collective success.
# Let me start with a story. Imagine a bustling city, where people pass each other on the streets without a second glance. Now, picture a different scenario: a small town where neighbors greet each other warmly, share stories, and lend a helping hand. Which community do you think is happier and more vibrant? The latter, of course. Why? Because human connection matters.
# So, what exactly is human connection? It's more than just casual interactions or digital friendships. Human connection is about forging meaningful relationships, understanding and empathizing with one another, and experiencing a sense of belonging. It's about creating bonds that transcend differences and make us feel alive. Now, let's explore the impact of human connection on our lives. Research shows that people with strong social connections tend to live longer, healthier lives. When we feel connected, our stress levels decrease, our immune systems strengthen, and we become more resilient in the face of challenges. Human connection is the secret ingredient that nourishes our well-being.
# In today's digital age, it's easy to get caught up in virtual interactions and forget the power of genuine human connection. While technology has brought us closer in many ways, it has also created a sense of disconnection. We may have thousands of online friends, but how many true connections do we have? It's time to prioritize quality over quantity.
# Now, the question arises: how can we build genuine connections in a world that often feels disconnected? It starts with being present and actively listening to others. We need to engage in meaningful conversations, ask open-ended questions, and show genuine interest in others' stories. It's about stepping outside our comfort zones and embracing vulnerability. When we prioritize human connection, something incredible happens: a ripple effect. Our positive interactions with others inspire them to do the same. We create a chain reaction of compassion, kindness, and understanding that spreads far beyond our immediate circle. Imagine the transformative power of millions of interconnected individuals making a difference.
# In conclusion, let's remember that we are wired for connection. It's in our DNA. By fostering genuine human connections, we can unlock our full potential and create a more compassionate, inclusive world. So, let's reach out, connect, and make a difference in each other's lives. Together, we can change the world."""

# # Tokenize the input text
# encoded_inputs = tokenizer.encode_plus(text, padding=True, truncation=True, return_tensors='pt')

# # Forward pass through the model
# outputs = my_model(**encoded_inputs)

# # Retrieve the predicted logits for NSP (next sentence prediction)
# nsp_logits = outputs.logits

# # Apply softmax to obtain the predicted probabilities
# import torch.nn.functional as F
# nsp_probabilities = F.softmax(nsp_logits, dim=1)

# # The shape of nsp_probabilities is (1, 2)
# # The first element in the second dimension represents the probability of the next sentence being a continuation
# # The second element represents the probability of the next sentence being randomly selected from the training data


In [20]:
import torch
from transformers import RobertaTokenizer, RobertaForSequenceClassification
import re

tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
# Load your fine-tuned model
saved_model_path = "saved_model_fold_1/final_model"
my_model = RobertaForSequenceClassification.from_pretrained(saved_model_path)

def get_nsp_prob(sentence_1, sentence_2):
    # 문장을 토큰화하고 인코딩합니다.
    inputs = tokenizer.encode_plus(
        sentence_1, 
        sentence_2, 
        add_special_tokens=True, 
        return_tensors='pt'
    )

    # 모델에 입력을 전달하고 출력을 가져옵니다.
    outputs = my_model(
        input_ids=inputs['input_ids'], 
        attention_mask=inputs['attention_mask']
    )

    # "이어지는 문장"에 대한 확률을 반환합니다.
    logits = outputs.logits
    nsp_prob = torch.softmax(logits, dim=1)
    return nsp_prob[:, 1].item()

input_file = './bert_train_project/test/input.txt'
output_file = './bert_train_project/test/output.txt'

# 입력 파일에서 문장 쌍을 읽어들입니다.
with open(input_file, 'r') as f:
    lines = f.readlines()

# 출력 파일을 생성합니다.
with open(output_file, 'w') as f:
    # 각 문장 쌍에 대해 NSP를 계산하고 출력 파일에 기록합니다.
    for line in lines:
        # 문장을 마침표나 느낌표로 구분해서 두 개의 문장으로 나눕니다.
        try:
            sentences = re.split('[.!?]', line.strip())
            sentence_1 = sentences[0].strip()
            sentence_2 = sentences[1].strip()
        except:
            pass

        # NSP를 계산하고 출력 파일에 기록합니다.
        nsp_prob = get_nsp_prob(sentence_1, sentence_2)
        f.write(f'{nsp_prob}\n')
